In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
from shapely.geometry import LineString
from linref.events.geometry import LineStringM, linemerge_m, prepare_chained_linestring_m
import shapely

l1 = shapely.from_wkt('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2)')
l2 = shapely.from_wkt('LINESTRING (3 2, 4 3, 5 3, 6 4, 7 5)')
l3 = shapely.from_wkt('LINESTRING (8 5, 8 6, 9 7, 10 8, 11 9)')
lm1 = LineStringM(l1, [1, 5, 20, 30, 40])
lm2 = LineStringM(l2, [40, 60, 70, 80, 90])
lm3 = LineStringM(l3, [100, 110, 120, 130, 140])

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
df_linear_1 = pd.DataFrame(index=[7, 6, 9, 4, 3, 2, 0, 1], data={
    'county': ['A', 'A', 'A', 'A', 'A' ,'B', 'B', 'B'],
    'route': ['Main St', 'Main St', 'Main St', '1st Ave', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 0.5, 1.1, 10.0, 12.0, 13.0, 13.1, 1.8],
    'end': [0.4, 1.1, 3.0, 12.0, 13.0, 13.1, 14.0, 3.0],
    'volume': [100, 200, 200, 400, 500, 150, 200, 800],
    'class': ['B', 'A', 'A', 'A', 'B', 'A', 'A', 'A'],
})
#df_linear_1 = pd.concat([df_linear_1] * 100, ignore_index=True)#.sort_values(['route', 'beg'])
df_linear_2 = pd.DataFrame({
    'route': ['Main St', 'Main St', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 1.0, 10.0, 13.0, 13.1, 2.2],
    'end': [1.0, 2.0, 12.0, 13.0, 14.0, 2.8],
    'street_type': ['A', 'B', 'A', 'A', 'B', 'A'],
})
#df_linear_2 = pd.concat([df_linear_2] * 100, ignore_index=True)#.sort_values(['route', 'beg'])

ed1 = EventsData(index=df_linear_1.index, groups=df_linear_1.route, begs=df_linear_1.beg, ends=df_linear_1.end, closed='left_mod')
ed2 = EventsData(index=df_linear_2.index, groups=df_linear_2.route, begs=df_linear_2.beg, ends=df_linear_2.end, closed='left_mod')
ed3 = EventsData(index=df_linear_1.index, groups=df_linear_1.route, locs=df_linear_1.beg)

In [3]:
import linref as lr
lr.set_default_lrs(key_col=['county', 'route'], loc_col='loc', beg_col='beg', end_col='end', geom_col='geometry', closed='left_mod')
df = pd.concat([df_linear_1] * 1, ignore_index=False)

In [4]:
dissolved, relation = df.lr.dissolve(sort=False, return_relation=True)

array([0, 1, 1, 2, 2, 3, 3, 4])

array([0, 1, 2, 3, 4, 5, 6, 7])

[array([7]), array([6, 9]), array([4, 3]), array([2, 0]), array([1])]

[array([0]), array([1, 2]), array([3, 4]), array([5, 6]), array([7])]

In [5]:
relation.intersect_data.todense()

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [6]:
df

,county,route,beg,end,volume,class
7,A,Main St,0.0,0.4,100,B
6,A,Main St,0.5,1.1,200,A
9,A,Main St,1.1,3.0,200,A
4,A,1st Ave,10.0,12.0,400,A
3,A,1st Ave,12.0,13.0,500,B
2,B,2nd Ave,13.0,13.1,150,A
0,B,2nd Ave,13.1,14.0,200,A
1,B,Sycamore School Rd,1.8,3.0,800,A


In [7]:
dissolved

,county,route,beg,end,dissolved_index
0,A,Main St,0.0,0.4,[7]
1,A,Main St,0.5,3.0,"[6, 9]"
2,A,1st Ave,10.0,13.0,"[4, 3]"
3,B,2nd Ave,13.0,14.0,"[2, 0]"
4,B,Sycamore School Rd,1.8,3.0,[1]


In [9]:
relation.value_counts(data=dissolved['route'].values, axis=0)

,Main St,1st Ave,2nd Ave,Sycamore School Rd
7,1.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0
